In [21]:
import pandas as pd
import numpy as np
import nibabel as nib
from os.path import join

from scipy.stats import ttest_ind


In [14]:
df_pacientes = pd.read_csv('Clasificados.csv')
df_atlas = pd.read_csv('Harvard_Oxford.csv')
dir_ = r'C:\Users\jesus\Work\MIA\Autopsia'

atlas_path = r'C:\Users\jesus\Work\repos\Master_IA_Sanidad\Modulo_3\Clases\Clase_Semana_29_CDSS\cortex_Harvard_Oxford_orig_atlas.nii.gz'
atlas_img = nib.load(atlas_path)
atlas_data = atlas_img.get_fdata()

In [15]:
df_pacientes

,Unnamed: 0,SUBJECT_ID,ORIG_GROUP,LAST_DIAGNOSIS,FDG_ID,FDG_AGE,AGE_DEATH,ALZHEIMER PATHOLOGY,LEWY PATHOLOGY,AD_Pos,Lewy_Pos,Grupo
0,0,003_S_4892,AD,AD,334485,75.3,76,3,0,1.0,0.0,AD
1,1,005_S_0448,MCI,AD,226091,90.6,93,1,2,0.0,1.0,LEWY
2,2,005_S_4910,AD,AD,347770,81.7,84,3,4,1.0,1.0,Mixed
3,3,006_S_4546,AD,AD,300098,71.2,75,3,2,1.0,1.0,Mixed
4,4,011_S_0008,CN,MCI,26885,85.6,95,1,3,0.0,1.0,LEWY
...,...,...,...,...,...,...,...,...,...,...,...,...
68,68,127_S_4500,AD,AD,417020,92.5,95,3,0,1.0,0.0,AD
69,69,127_S_4765,EMCI,AD,871795,81.1,81,3,3,1.0,1.0,Mixed
70,70,127_S_5058,AD,AD,360030,61.9,66,2,3,1.0,1.0,Mixed
71,71,129_S_1246,MCI,MCI,362092,78.5,80,2,0,1.0,0.0,AD


In [18]:
for index_1, row_1 in df_pacientes.iterrows():
    
    id_subject = row_1['SUBJECT_ID']
    id_img = row_1['FDG_ID']
    subj_dir_name = id_subject + '_I' + str(id_img)

    img_path = join(dir_,subj_dir_name,'swfdg_normhist.hdr')
    img_ = nib.load(img_path)
    img_data = img_.get_fdata()
    
    for index_2, row_2 in df_atlas.iterrows():
        roi_name = row_2['ROI_NAME']
        roi_num = row_2['ROI_NUM']
        roi_promedio = row_2['PROMEDIO']
        roi_std = row_2['STD']
        
        roi_voxel = np.where(atlas_data == roi_num)
        val_patient = np.mean(img_data[roi_voxel])
        
        pat_roi_zscore = (val_patient - roi_promedio)/roi_std

        df_pacientes.loc[index_1,roi_name] = pat_roi_zscore
                

In [24]:
df_pacientes

,Unnamed: 0,SUBJECT_ID,ORIG_GROUP,LAST_DIAGNOSIS,FDG_ID,FDG_AGE,AGE_DEATH,ALZHEIMER PATHOLOGY,LEWY PATHOLOGY,AD_Pos,...,Planum_Polare,Heschl's_Gyrus,Planum_Temporale,Supracalcarine_Cortex,Occipital_Pole,Thalamus,Striatum,Brainstem,Hippocampus,Amygdala
0,0,003_S_4892,AD,AD,334485,75.3,76,3,0,1.0,...,-0.112321,-0.958489,-1.922645,2.093158,0.357431,0.006967,-0.296966,-0.278695,-0.094379,-0.625237
1,1,005_S_0448,MCI,AD,226091,90.6,93,1,2,0.0,...,-0.080783,0.581598,0.430449,-1.002594,-2.052067,-1.566412,1.811901,1.976478,0.854247,1.853367
2,2,005_S_4910,AD,AD,347770,81.7,84,3,4,1.0,...,-3.597921,-2.232594,-2.430986,-0.353698,-0.505277,-1.481930,-2.080433,0.162197,-2.105653,-1.938893
3,3,006_S_4546,AD,AD,300098,71.2,75,3,2,1.0,...,0.937891,1.258248,0.175496,2.846217,1.965820,-1.520694,-1.832788,0.432951,-0.682365,0.846385
4,4,011_S_0008,CN,MCI,26885,85.6,95,1,3,0.0,...,1.075426,0.278105,-0.128056,-0.168973,-0.532507,0.968607,0.803972,0.301074,0.950026,0.608191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,68,127_S_4500,AD,AD,417020,92.5,95,3,0,1.0,...,0.745240,-0.208900,-0.339694,-2.179645,-1.124119,-1.936402,-1.862072,0.079512,0.819908,1.461137
69,69,127_S_4765,EMCI,AD,871795,81.1,81,3,3,1.0,...,1.722327,-0.339546,-0.589497,-1.592252,-3.282584,1.862965,1.764669,1.218454,1.584905,1.544832
70,70,127_S_5058,AD,AD,360030,61.9,66,2,3,1.0,...,2.331087,0.977442,0.349620,0.075130,0.411157,-0.471721,-0.887564,-0.471022,0.377695,0.904273
71,71,129_S_1246,MCI,MCI,362092,78.5,80,2,0,1.0,...,-1.454552,-0.287481,-0.080177,0.642108,0.269712,-0.247182,1.028909,-1.428884,-0.627655,-0.710637


In [27]:
df_ad = df_pacientes.loc[df_pacientes['Grupo']=='AD']
df_lewy = df_pacientes.loc[df_pacientes['Grupo']=='LEWY']

ad_pattern = np.zeros_like(atlas_data)
lewy_body_pattern = np.zeros_like(atlas_data)

for index_2, row_2 in df_atlas.iterrows():
    roi_num = row_2['ROI_NUM']
    roi_name = row_2['ROI_NAME']
    
    mean_ad = np.mean(df_ad[roi_name])
    mean_lewy = np.mean(df_lewy[roi_name])
    
    roi_voxels = np.where(atlas_data == roi_num)

    ad_pattern[roi_voxels] = mean_ad
    lewy_body_pattern[roi_voxels] = mean_lewy


ad_pat_img = nib.Nifti1Image(ad_pattern, atlas_img.affine, atlas_img.header)
nib.save(ad_pat_img, 'ad_pattern.nii.gz')

lewy_pat_img = nib.Nifti1Image(lewy_body_pattern, atlas_img.affine, atlas_img.header)
nib.save(lewy_pat_img, 'lewy_pattern.nii.gz')
    
    


In [25]:
df_ad = df_pacientes.loc[df_pacientes['Grupo']=='AD']
df_lewy = df_pacientes.loc[df_pacientes['Grupo']=='LEWY']

roi_list = []

for index_2, row_2 in df_atlas.iterrows():

    roi_name = row_2['ROI_NAME']
    t, p = ttest_ind(df_ad[roi_name], df_lewy[roi_name], axis=0, equal_var=True)
    if p < 0.05:
        print(roi_name, t, p)
        roi_list.append(roi_name)

Temporal_Pole -2.9044114080274825 0.006423868599824267
Middle_Temporal_Gyrus_anterior_division -2.9449405299732736 0.005791246618907714
Inferior_Temporal_Gyrus_anterior_division -3.648013636244977 0.0008766686428361144
Inferior_Temporal_Gyrus_posterior_division -2.2423207698684227 0.03157198455754205
Postcentral_Gyrus 2.339018827579624 0.025349047504676273
Superior_Parietal_Lobule 2.0596797178601154 0.047151216361886496
Intracalcarine_Cortex 3.4228607548346344 0.0016316623398195012
Frontal_Medial_Cortex -2.780857134284928 0.008775985499674939
Subcallosal_Cortex -2.379668508133448 0.023081468186700788
Cuneal_Cortex 2.3451998633745976 0.02499174350658922
Frontal_Orbital_Cortex -2.1124984197960734 0.04206588756594934
Lingual_Gyrus 2.1627040348763056 0.037687823572323115
Supracalcarine_Cortex 3.0143807283736326 0.004841642830535394
Occipital_Pole 2.366443565140639 0.023798180975778616


In [26]:
len(roi_list)

14